In [1]:
# libs for detector
import random
from detectron2.utils.visualizer import Visualizer
from detectron2.data.catalog import MetadataCatalog, DatasetCatalog
import time
import cv2
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2 import model_zoo
import os
from detectron2.engine.defaults import DefaultPredictor
from detectron2.utils.visualizer import ColorMode
from detectron2.data.datasets import register_coco_instances
from detectron2.structures import Instances
import torch 

# %pylab inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import time

In [21]:
# CATEGORIES =  ['stop', 'yield', 'do_not_enter', 'other_regulatory', 'other_prohibitory', 'warning_pedestrians']

class DynamicDetectorEngine:
    def __init__(self):
#         MetadataCatalog.get("empty_dataset").thing_classes = CATEGORIES
#         self.mapillary_metadata = MetadataCatalog.get("empty_dataset")
        
        cfg = get_cfg()
        cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
        cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.80 # set threshold for this model
        cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
        self.metadata = MetadataCatalog.get(cfg.DATASETS.TRAIN[0])
        
        self.predictor = DefaultPredictor(cfg)

        
        
    def inference(self, img, counter):
        show_flag = False
        save_flag = False
        outputs = self.predictor(img) 
        if len(outputs["instances"]):
            # Instances Class: read https://detectron2.readthedocs.io/tutorials/models.html#model-output-format
            outputs = outputs["instances"].to("cpu") 
#             cls = outputs.pred_classes.numpy()
#             scores = outputs.scores.numpy()
#             bboxes = outputs.pred_boxes.tensor.numpy()
#             masks = outputs.pred_masks.numpy()
    
            v = Visualizer(img,
                        metadata=self.metadata, 
                        scale=0.8, 
                        instance_mode=ColorMode.IMAGE   # remove the colors of unsegmented pixels
            )
            v = v.draw_instance_predictions(outputs)
            if show_flag:
#                 cv2.namedWindow('Sign Detector', 0)
#                 cv2.imshow('Sign Detector', v.get_image()[:, :, ::-1])
#                 cv2.waitKey(0)
                imgplot = plt.imshow(v.get_image())
                plt.show()
            if save_flag:
                img_name = "/home/ubuntu/albert/results/maskrcnn/frame_{}.jpg".format(counter)
                cv2.imwrite(img_name, v.get_image()[:, :, ::-1])
            
                
        return outputs

In [22]:
model = DynamicDetectorEngine()

[12/11/2020 21:08:05 INFO] Loading checkpoint from https://dl.fbaipublicfiles.com/detectron2/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/137849600/model_final_f10217.pkl
[12/11/2020 21:08:05 INFO] Reading a file from 'Detectron2 Model Zoo'


In [21]:
import glob
from tqdm import tqdm

img_dir = '/home/ubuntu/albert/datasets/argoverse-tracking/train1/043aeba7-14e5-3cde-8a5c-639389b6d3a6/ring_front_center/'
counter = 0
for img_name in tqdm(glob.glob(img_dir + '*.jpg')):
    counter += 1
    img = mpimg.imread(img_name)
    output = model.inference(img, counter)
#     break
#     print(output)

100%|██████████| 469/469 [03:19<00:00,  2.35it/s]


## Read 3D model from files

In [55]:
import numpy as np
import cv2
from hloc.utils.read_write_model import read_images_binary, read_points3d_binary, write_images_binary, write_points3d_binary

In [64]:
# model_dir = '/home/ubuntu/albert/workspace/Hierarchical-Localization/outputs/argo_sample_2/sfm_superpoint+superglue+netvlad/models/model_aligned/'
model_dir = '/home/ubuntu/albert/workspace/Hierarchical-Localization/outputs/argo_sample_2/sfm_superpoint+superglue+netvlad/models/0/'
images = read_images_binary(model_dir + 'images.bin')
points3D = read_points3d_binary(model_dir + 'points3D.bin')
print(len(list(images.keys())))
print(len(list(points3D.keys())))

785
110001


## Iterate the database image 

In [65]:
import numpy as np
def in_mask(feature_position, masks, classes):
    for i in range(masks.shape[0]):
        cls = classes[i]
        if cls in list(range(10)):
            mask = masks[i]
            pos_round = np.round(feature_position).astype(np.int32)
            if mask[pos_round[1], pos_round[0]]:
                return True
    
    return False

In [66]:
from tqdm import tqdm
db_img_dir = '/home/ubuntu/albert/datasets/argoverse-tracking/sample/c6911883-1843-3727-8eaa-41dc8cda8993/'
counter = 0

for img_item in tqdm(list(images.values())):
    
    image_name = img_item.name
    image_id = img_item.id
    keypoints = img_item.xys
    #     pt3d_ids = img_item.point3D_ids
    
    camera_type = image_name.split('_')[:3]
    camera_type = '_'.join(camera_type)
    img = mpimg.imread(db_img_dir + camera_type + '/' + image_name)
    outputs = model.inference(img, 0)
#     print(outputs)

    cls = outputs.pred_classes.numpy()
    scores = outputs.scores.numpy()
    bboxes = outputs.pred_boxes.tensor.numpy()
    masks = outputs.pred_masks.numpy()
    
#     print(cls)
#     print(masks.shape)                
    print('Found {} instances in image {}'.format(cls.shape[0], image_name))
    for i in range(keypoints.shape[0]):
        if img_item.point3D_ids[i] != -1:
            if in_mask(keypoints[i], masks, cls):
                pt3d_id = img_item.point3D_ids[i]
                # find the images containing the target 3d point
                delete_image_ids = points3D[pt3d_id].image_ids
                delete_keypoint_ids = points3D[pt3d_id].point2D_idxs
                
                for j, delete_img_id in enumerate(list(delete_image_ids)):
                    delete_keypoint_id = delete_keypoint_ids[j]
                    images[delete_img_id].point3D_ids[delete_keypoint_id] = -1                  
                
                points3D.pop(pt3d_id)
                counter += 1
    print('{} keypoints deleted in total\n'.format(counter))
#     break

print(len(list(points3D.keys())))    

  0%|          | 1/785 [00:00<04:54,  2.66it/s]

Found 11 instances in image ring_front_center_315978406032859416.jpg
96 keypoints deleted in total



  0%|          | 2/785 [00:00<04:48,  2.71it/s]

Found 11 instances in image ring_front_center_315978406066159560.jpg
122 keypoints deleted in total



  0%|          | 3/785 [00:01<04:48,  2.71it/s]

...

100%|█████████▉| 784/785 [04:01<00:00,  2.91it/s]

Found 9 instances in image ring_rear_right_315978421517353384.jpg
22839 keypoints deleted in total



100%|██████████| 785/785 [04:02<00:00,  3.24it/s]

Found 9 instances in image ring_rear_right_315978421550653552.jpg
22846 keypoints deleted in total

87155


New model saved


In [ ]:
# import os

# output_dir_db = '/home/ubuntu/albert/workspace/Hierarchical-Localization/outputs/argo_sample_2/sfm_superpoint+superglue+netvlad/models/filtered/'
# if not os.path.exists(output_dir_db):
#     os.makedirs(output_dir_db)
    
write_images_binary(images, model_dir + 'images.bin')
write_points3d_binary(points3D, model_dir + 'points3D.bin')
print('New model saved')


In [63]:
images = read_images_binary(model_dir + 'images_filtered.bin')
points3D = read_points3d_binary(model_dir + 'points3D_filtered.bin')
print(len(list(images.keys())))
print(len(list(points3D.keys())))

785
87155


## To get the original database image set for the model

In [68]:
import os

image_folder = '/home/ubuntu/albert/datasets/argoverse-tracking/sample/c6911883-1843-3727-8eaa-41dc8cda8993/'
output_dir_db = '/home/ubuntu/albert/datasets/argoverse-tracking/sample/test_1017_2/db/'
if not os.path.exists(output_dir_db):
    os.makedirs(output_dir_db)
output_dir_query = '/home/ubuntu/albert/datasets/argoverse-tracking/sample/test_1017_2/query/'
if not os.path.exists(output_dir_query):
    os.makedirs(output_dir_query)

for img_item in tqdm(list(images.values())):
    
    image_name = img_item.name
    camera_type = image_name.split('_')[:3]
    camera_type = '_'.join(camera_type)
    filename = image_folder + camera_type + '/' + image_name
    cmd = 'cp '+ filename + ' ' + output_dir_db
    os.system(cmd)
            
output_dir_db_query = '/home/ubuntu/albert/datasets/argoverse-tracking/sample/test_1017_2/db_query/'
if not os.path.exists(output_dir_db_query):
    os.makedirs(output_dir_db_query)
cmd = 'cp '+ output_dir_db + '/*.jpg ' + output_dir_db_query
os.system(cmd)
cmd = 'cp '+ output_dir_query + '/*.jpg ' + output_dir_db_query
os.system(cmd)

100%|██████████| 785/785 [01:20<00:00,  9.70it/s]


256